<a href="https://colab.research.google.com/github/7ft10/JiraExporter/blob/main/DataFlows - Server/Jira_Epics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#! Library Import

import os 
import importlib.util

if importlib.util.find_spec("pandas") is None:	
	os.system("pip install pandas")

if importlib.util.find_spec("dotenv") is None:	
	os.system("pip install --quiet openai python-dotenv")

In [3]:
#! pandas Config

import pandas as pd
import requests
requests.packages.urllib3.disable_warnings(requests.packages.urllib3.exceptions.InsecureRequestWarning)

pd.options.mode.use_inf_as_na = True
#pd.set_option("display.max_rows", 10)
#pd.set_option("display.expand_frame_repr", True)
#pd.set_option('display.width', 1000)
#pd.options.display.max_seq_items = 200000
#pd.options.display.max_rows = 10

In [4]:
#! Parameters

import os 
import dotenv
import importlib.util

if importlib.util.find_spec("google.colab") is not None: ## if using google colab
    if not os.path.exists('.env'):
        from google.colab import files
        uploaded = files.upload()
        file_name = list(uploaded.keys())[0]
        try:
            os.rename(file_name, '.env')
        except:
            pass

dotenv.load_dotenv('.env')

Host = os.getenv('SECRETS_HOST')
Username = os.getenv('SECRETS_USERNAME')
Password = os.getenv('SECRETS_PASSWORD')
ValidProjectCategories = ["'Portfolio'"]

if Host is None:
    raise Exception("Secrets not found")

display("Host: " + Host)
display("ValidProjectCategories: " + ','.join(ValidProjectCategories))

'Host: https://jira.budgetdirect.com.au/'

"ValidProjectCategories: 'Portfolio'"

In [5]:
#! Functions

import base64
import pandas as pd
import re 
import requests
import warnings
from functools import reduce
    
def _ExpandColumn(self:pd.DataFrame, colName:str, columnsToExpand = [], prefix:str = "Prefix", sentenceCase:bool = True) -> pd.DataFrame:
    if (prefix == "Prefix"):
        prefix = colName + " "
        with warnings.catch_warnings():
          warnings.simplefilter(action='ignore', category=FutureWarning)
          expandedCols = self[colName].apply(lambda x: pd.Series(x).add_prefix(prefix))
        columnsToExpand = [prefix + c for c in columnsToExpand]
    else:
        expandedCols = self[colName].apply(lambda x: pd.Series(x))
    
    if len(columnsToExpand) > 0:        
        expandedCols = expandedCols[columnsToExpand]
    
    if sentenceCase:
        expandedCols.columns = [fnSentenceCase(c) for c in expandedCols.columns] 

    return pd.concat([self.drop(colName, axis=1), expandedCols], axis=1)

pd.DataFrame.expand = _ExpandColumn

def fnSentenceCase(s):
    s = (' '.join(dict.fromkeys(s.split())))  # remove duplicate words
    s = s.replace("0", "") # remove "0" 
    s = s.strip()
    return ' '.join([x.capitalize() for x in re.sub(r"([A-Z])", r" \1", s).split()]) # sentence case

def _SentenceCaseColumns(self:pd.DataFrame) -> pd.DataFrame: 
    self.columns = [fnSentenceCase(c) for c in self.columns] 
    return self

pd.DataFrame.sentence_case_columns = _SentenceCaseColumns

def fnGetDefaultHeaders():
    return {
        "content-type": "application/json",
        "authorization": "Basic " + base64.b64encode((Username + ":" + Password).encode()).decode(),
        "retry-after": "120"
    }

def fnSearch(jql, fields = None, expand = None):
    def ApiCall(startAt) :
        url = "/rest/api/latest/search"
        headers = fnGetDefaultHeaders()
        defaultContents = {
            "startAt": startAt,
            "maxResults": "2",
            "jql": jql
        }
        if fields is not None:
            defaultContents["fields"] = fields.tolist()
        if expand is not None and expand != "":
            defaultContents["expand"] = expand        
        response = requests.post(Host + url, headers = headers, json = defaultContents, verify=False)
        return response.json()
    values = fnAPI(ApiCall)
    if len(values.index) > 1:
        return values
    else:
        None

def fnGetIssueTypeFields(IssueTypes) -> pd.DataFrame:
    def ApiCall(startAt) :
        url = "rest/api/latest/issue/createmeta"
        headers = fnGetDefaultHeaders()
        params = {
            "expand": "projects.issuetypes.fields",
            "projectKeys": ','.join(fnGetValidProjectKeys()["key"].values),
            "issuetypeNames": ','.join(IssueTypes).replace("'", "")
        }
        response = requests.get(Host + url, headers = headers, params = params, verify=False)
        return response.json()
    
    df = fnAPI(ApiCall)
    df = df.drop(["expand"], axis=1)
    df = df.explode("projects")
    df = df.expand("projects", [], None, False)    
    try:
        df = df[["issuetypes"]]
    except: 
        raise Exception("No issue metadata - check the valid project categories are correct")
    df = df.explode("issuetypes")
    df = df.expand("issuetypes", [], None, False)
    df = df[["fields"]]
    df = df.expand("fields", [], None, False)
    df = df.loc[:,~df.columns.duplicated()]

    values = []
    for x in df.columns:        
        try:            
            valid:pd.DataFrame = pd.DataFrame( df[~df[x].isnull()] )[[x]].iloc[0].get(0)
            values.append({
                "fieldId": valid['key'] if "key" in valid else valid["fieldId"],
                "name": valid['name'],
                "schema_type": valid['schema']['type'],
                "required": valid['required']
            })
        except:
            #display(x)
            pass
    values.append({ "fieldId": 'status', "name": 'Status', "schema_type": 'string', "required": True })
    values.append({ "fieldId": 'created', "name": 'Created', "schema_type": 'date', "required": True })
    values.append({ "fieldId": 'updated', "name": 'Updated', "schema_type": 'date', "required": True })
    values.append({ "fieldId": 'resolution', "name": 'Resolution', "schema_type": 'string', "required": True })
    values.append({ "fieldId": 'resolutiondate', "name": 'Resolution Date', "schema_type": 'date', "required": False })
    values.append({ "fieldId": 'lastViewed', "name": 'Last Viewed', "schema_type": 'date', "required": True })
    values.append({ "fieldId": 'id', "name": 'Id', "schema_type": 'number', "required": True })
    values.append({ "fieldId": 'key', "name": 'Key', "schema_type": 'string', "required": True })
    df = pd.DataFrame(values)
    df = df.drop_duplicates().sort_values("fieldId")
    return df 

def fnGetValidProjectKeys() -> pd.DataFrame:
    def ApiCall(startAt) :
        url = "/rest/api/latest/project"
        headers = fnGetDefaultHeaders()
        params = { }
        response = requests.get(Host + url, headers = headers, params = params, verify=False)
        return response.json()
    df = fnAPI(ApiCall)
    df = df.expand("projectCategory")
    if 'ValidProjectCategories' in globals() and len(ValidProjectCategories) > 0:
        df = df.loc[df['Project Category Name'].isin(ValidProjectCategories) | ("'" + df['Project Category Name'] + "'").isin(ValidProjectCategories)]
    return df[["key"]]

def fnAPI(webRequestDelegate, startAt = 0) -> pd.DataFrame:
    def flatten_reduce_lambda(frm):
        try:
            return list(reduce(lambda x, y: x + y, frm, []))         
        except:
            return list(reduce(lambda x, y: x + y, [frm], [])) 
    def innerGetResults(webRequestDelegate, startAt = 0):
        results = webRequestDelegate(startAt)
        if isinstance(results, dict) and "total" in results and "maxResults" in results:
            if startAt + results["maxResults"] < results["total"]:
                return [results] + innerGetResults(webRequestDelegate, startAt + results["maxResults"])
            else:
                return [results]
        else:
            return [results]
    Source = flatten_reduce_lambda(innerGetResults(webRequestDelegate, startAt))
    df = pd.DataFrame(Source)
    return df


In [6]:
#! Jira Issues Capture

from IPython.display import display

ExpectedIssueTypes = [ "Epic" ]
JQL = "issuetype in (" + ','.join(ExpectedIssueTypes) + ") and category in (" + ','.join(ValidProjectCategories) + ") ORDER BY updatedDate DESC"

fields = fnGetIssueTypeFields(ExpectedIssueTypes)

goldenDF = None
globals()['goldenDF'] = None 

df = fnSearch(JQL, fields["fieldId"].values)
if df is None:	
	display("No results")
else:
	df = df.drop(["expand", "startAt", "maxResults", "total"], axis=1)
	df = df.explode("issues")
	df = df.expand("issues", [], None, False)
	df = df.drop(["expand", "self"], axis=1)
	df = df.expand("fields", [], None, False)		

	df = df.rename( columns=dict( zip ( fields.fieldId, fields.name )) )	

	df = df.convert_dtypes().infer_objects().reset_index(drop=True)
	df["Id"] = df["Id"].astype('Int64')

	goldenDF:pd.DataFrame = df.copy(deep = True)
	globals()['goldenDF'] = goldenDF # make this globally available 

if 'goldenDF' not in globals() or goldenDF is None: 
	display("Base data frame not loaded") 
else:	
	display(goldenDF.dtypes)
	display(goldenDF.head())

Id                              Int64
Key                    string[python]
Issue Type                     object
Risk Probability               object
Acceptance Criteria            object
Component/s                    object
Change is                      object
Sprint                         object
Epic Name              string[python]
Epic Link                      object
Requirement Status             object
Description            string[python]
Project                        object
T-Shirt Size                   object
Fix Version/s                  object
Resolution                     object
Attachment                     object
Resolution Date        string[python]
Summary                string[python]
Last Viewed                    object
Delivery Teams                 object
Created                string[python]
Reporter                       object
Priority                       object
Labels                         object
Story Points                    Int64
Linked Issue

In [7]:
#! Issues

from IPython.display import display

if 'goldenDF' not in globals() or goldenDF is None: 
   display("Base data frame not loaded") 
else:
   df:pd.DataFrame = goldenDF.copy(deep = True)
   if df is None or len(df.index) == 0:	
      display("No results")
   else:
      df = df.drop(["Component/s", "Attachment", "Linked Issues", "Sprint", "Fix Version/s", "Labels"], axis=1)
      df = df.expand("Issue Type", ["id", "name"])
      df = df.expand("Status", ["id", "name", "statusCategory"]).sentence_case_columns()
      df = df.expand("Status Category", ["id", "name"]) 
      df = df.expand("Project", ["id", "name", "projectCategory"]).sentence_case_columns()
      df = df.expand("Project Category", ["id", "name"])
      df = df.expand("Reporter", ["key", "displayName"]) #df = df.expand("Reporter", ["accountId", "displayName"]) 
      df = df.expand("Assignee", ["key", "displayName"]) #df = df.expand("Assignee", ["accountId", "displayName"])
      df = df.expand("Priority", ["id", "name"])
      df = df.expand("Resolution", ["id", "name"])
      #df = df.expand("Parent", ["id", "key"])     
      #df = df.expand("Account", ["id", "value"])   
      #df = df.expand("Tempo Customer", ["id", "value"])   

      df = df.sentence_case_columns().convert_dtypes().infer_objects().reset_index(drop=True)
      df["Story Points"] = df["Story Points"].astype('float')
      
      display(df.dtypes)
      display(df.head())

Id                                Int64
Key                      string[python]
Risk Probability                 object
Acceptance Criteria              object
Change Is                        object
Epic Name                string[python]
Epic Link                        object
Requirement Status               object
Description              string[python]
T- Shirt Size                    object
Resolution Date          string[python]
Summary                  string[python]
Last Viewed                      object
Delivery Teams                   object
Created                  string[python]
Story Points                    float64
Updated                  string[python]
Issue Type Id            string[python]
Issue Type Name          string[python]
Status Id                string[python]
Status Name              string[python]
Status Category Id                Int64
Status Category Name     string[python]
Project Id               string[python]
Project Name             string[python]


,Id,Key,Risk Probability,Acceptance Criteria,Change Is,Epic Name,Epic Link,Requirement Status,Description,T- Shirt Size,...,Project Category Id,Project Category Name,Reporter Key,Reporter Display Name,Assignee Key,Assignee Display Name,Priority Id,Priority Name,Resolution Id,Resolution Name
0,630363,PPI-1420,{'self': 'https://jira.budgetdirect.com.au/res...,None,{'self': 'https://jira.budgetdirect.com.au/res...,Test Academy - Functional Testing,None,[],<NA>,None,...,10930,Portfolio,JIRAUSER32846,Shaun Loh,<NA>,<NA>,4,Minor,<NA>,<NA>
1,631352,PPI-1429,{'self': 'https://jira.budgetdirect.com.au/res...,None,{'self': 'https://jira.budgetdirect.com.au/res...,TCM Enhancements,None,[],<NA>,None,...,10930,Portfolio,JIRAUSER32020,Angela Baro,JIRAUSER32020,Angela Baro,4,Minor,<NA>,<NA>
2,630338,PPI-1402,{'self': 'https://jira.budgetdirect.com.au/res...,None,{'self': 'https://jira.budgetdirect.com.au/res...,Test Reporting - Functional,None,[],<NA>,None,...,10930,Portfolio,JIRAUSER32846,Shaun Loh,<NA>,<NA>,4,Minor,<NA>,<NA>
3,648407,TAP-219,{'self': 'https://jira.budgetdirect.com.au/res...,None,{'self': 'https://jira.budgetdirect.com.au/res...,Team Admin Tasks,None,[],<NA>,None,...,10930,Portfolio,JIRAUSER32846,Shaun Loh,<NA>,<NA>,4,Minor,<NA>,<NA>
4,642603,TAP-75,{'self': 'https://jira.budgetdirect.com.au/res...,None,{'self': 'https://jira.budgetdirect.com.au/res...,OPM Regression Suite,None,[],<NA>,None,...,10930,Portfolio,JIRAUSER32846,Shaun Loh,<NA>,<NA>,4,Minor,<NA>,<NA>


In [8]:
#! Components

from IPython.display import display

from sqlalchemy import create_engine
  
# Create the engine to connect to the inbuilt sqllite database
engine = create_engine("sqlite+pysqlite:///:memory:")

if 'goldenDF' not in globals() or goldenDF is None: 
    display("Base data frame not loaded") 
else:
    df:pd.DataFrame = goldenDF.copy(deep = True)
    df = df[["Id", "Key", "Component/s"]]
    df = df[df["Component/s"].map(lambda d: len(d)) > 0]

    if df is None or len(df.index) == 0:	
        display("No results")
    else:
        df = df.explode("Component/s")
        df = df.expand("Component/s", ["id", "name"])

        df = df.sentence_case_columns().convert_dtypes().infer_objects().reset_index(drop=True)

        display(df.dtypes)
        display(df.head())

Id                           Int64
Key                 string[python]
Component/s Id      string[python]
Component/s Name    string[python]
dtype: object

,Id,Key,Component/s Id,Component/s Name
0,632683,PPI-1452,26922,Clouding_Atlassian
1,632684,PPI-1453,26922,Clouding_Atlassian
2,630285,PPI-1361,26445,PPI Operational
3,632678,PPI-1449,26922,Clouding_Atlassian
4,652670,PPI-1711,26444,Dynamic Delivery


In [107]:
#! Linked Issues

from IPython.display import display

if 'goldenDF' not in globals() or goldenDF is None: 
    display("Base data frame not loaded") 
else:
    df:pd.DataFrame = goldenDF.copy(deep = True)
    df = df[["Id", "Key", "Linked Issues"]]
    df = df[df["Linked Issues"].map(lambda d: len(d)) > 0]

    if df is None or len(df.index) == 0:
        display("No results")
    else:
        df = df.explode("Linked Issues")
        
        def transform_row(r):
            linkedIssue = r.get("Linked Issues")
            inward = "inwardIssue" in linkedIssue.keys()
            r["Type Id"] = linkedIssue["type"]["id"]
            r["Type Name"] = linkedIssue["type"]["name"]
            r["Type Direction"] = "Inward" if inward else "Outward"
            r["Type Description"] = linkedIssue["type"]["inward"] if inward else linkedIssue["type"]["outward"]
            r["Linked Issue"] = linkedIssue["inwardIssue"] if inward else linkedIssue["outwardIssue"]
            return r

        df = df.apply(transform_row, axis=1)
        df = df.drop(["Linked Issues"], axis=1)
        df = df.expand("Linked Issue", ["id", "key"])

        df = df.sentence_case_columns().convert_dtypes().infer_objects().reset_index(drop=True)
        df["Type Id"] = df["Type Id"].astype('Int64')
        df["Linked Issue Id"] = df["Linked Issue Id"].astype('Int64')

        display(df.dtypes)
        display(df.head())

Id                           Int64
Key                 string[python]
Type Id                      Int64
Type Name           string[python]
Type Direction      string[python]
Type Description    string[python]
Linked Issue Id              Int64
Linked Issue Key    string[python]
dtype: object

,Id,Key,Type Id,Type Name,Type Direction,Type Description,Linked Issue Id,Linked Issue Key
0,630363,PPI-1420,10450,Child-Issue,Inward,is child task of,630401,AGP-5760
1,630338,PPI-1402,10450,Child-Issue,Inward,is child task of,630399,AGP-5758
2,641726,FLOW-392,10030,Cloners,Outward,Clones,633531,FLOW-175
3,632683,PPI-1452,10450,Child-Issue,Inward,is child task of,526026,AGP-3267
4,632683,PPI-1452,10650,Related Issues,Outward,relates to,632679,PPI-1450


In [10]:
#! Sprints

from IPython.display import display
from dateutil import parser
from pandas.core.dtypes.dtypes import DatetimeTZDtype

if 'goldenDF' not in globals() or goldenDF is None: 
    display("Base data frame not loaded") 
else:
    df:pd.DataFrame = goldenDF.copy(deep = True)
    df = df[["Id", "Key", "Sprint"]]
    df = df[~df["Sprint"].isna()]

    if df is None or len(df.index) == 0:	
        display("No results")
    else:		
        df = df.explode("Sprint")
        df = df.expand("Sprint")

        if "Sprint Start Date" not in df.columns.values:
            # Server instance not cloud 
            def fixSprint(s):
                try:
                    s = (s.split('[', 1)[1])[:-1]
                    res = []
                    for sub in s.split(','):
                        if '=' in sub:
                            res.append(map(str.strip, sub.split('=', 1)))
                    return dict(res)
                except: 
                    return None
            df["Sprint"] = df["Sprint"].apply (fixSprint)
            df = df.expand("Sprint")

            def fixDate(s):
                try:
                    return parser.isoparse(s)
                except:
                    return None
            df["Sprint Start Date"] = df["Sprint Start Date"].apply(fixDate)
            df["Sprint End Date"] = df["Sprint End Date"].apply(fixDate)
            df["Sprint Complete Date"] = df["Sprint Complete Date"].apply(fixDate)
            
        df = df.sentence_case_columns().convert_dtypes().infer_objects().reset_index(drop=True)

        df["Sprint Start Date"] = df["Sprint Start Date"].astype(DatetimeTZDtype("ns", "+10:00"))
        df["Sprint End Date"] = df["Sprint End Date"].astype(DatetimeTZDtype("ns", "+10:00"))
        df["Sprint Complete Date"] = df["Sprint Complete Date"].astype(DatetimeTZDtype("ns", "+10:00"))
        
        display(df.dtypes)
        display(df.head())


Id                                            Int64
Key                                  string[python]
Sprint Id                            string[python]
Sprint Rapid View Id                 string[python]
Sprint State                         string[python]
Sprint Name                          string[python]
Sprint Start Date         datetime64[ns, UTC+10:00]
Sprint End Date           datetime64[ns, UTC+10:00]
Sprint Complete Date      datetime64[ns, UTC+10:00]
Sprint Activated Date                string[python]
Sprint Sequence                      string[python]
Sprint Goal                          string[python]
Sprint Auto Start Stop               string[python]
dtype: object

,Id,Key,Sprint Id,Sprint Rapid View Id,Sprint State,Sprint Name,Sprint Start Date,Sprint End Date,Sprint Complete Date,Sprint Activated Date,Sprint Sequence,Sprint Goal,Sprint Auto Start Stop
0,633036,FLOW-169,5514,1116,CLOSED,PPI-FY23Q3-S3,2023-02-06 13:54:00+10:00,2023-02-17 13:54:00+10:00,2023-02-20 14:23:01.092000+10:00,2023-02-06T11:04:28.030+10:00,5514,(1) Finalise and publish changes in response t...,false
1,633531,FLOW-175,5495,1116,CLOSED,PPI-FY23Q2-S7,2022-12-12 11:00:00+10:00,2022-12-23 17:00:00+10:00,2023-01-09 09:26:05.242000+10:00,2022-12-12T10:51:45.687+10:00,5495,(1) FLOW 69-70; (2) Clouding Atlassian Impleme...,false


In [11]:
#! Attachments

from IPython.display import display
from pandas.core.dtypes.dtypes import DatetimeTZDtype

if 'goldenDF' not in globals() or goldenDF is None: 
    display("Base data frame not loaded") 
else:
    df:pd.DataFrame = goldenDF.copy(deep = True)
    df = df[["Id", "Key", "Attachment"]]
    df = df[df["Attachment"].map(lambda d: len(d)) > 0]

    if df is None or len(df.index) == 0:	
        display("No results")
    else:
        df = df.explode("Attachment")
        df = df.expand("Attachment", ["id", "filename", "created", "mimeType", "size", "thumbnail", "content", "author"])
        df = df.expand("Attachment Author", ["key"])

        df = df.sentence_case_columns().convert_dtypes().infer_objects().reset_index(drop=True)
        df["Attachment Id"] = df["Attachment Id"].astype('Int64')
        df["Attachment Size"] = df["Attachment Size"].astype('float64')
        df["Attachment Created"] = df["Attachment Created"].astype(DatetimeTZDtype("ns", "+10:00"))

        display(df.dtypes)
        display(df.head())

Id                                           Int64
Key                                 string[python]
Attachment Id                                Int64
Attachment Filename                 string[python]
Attachment Created       datetime64[ns, UTC+10:00]
Attachment Mime Type                string[python]
Attachment Size                            float64
Attachment Thumbnail                string[python]
Attachment Content                  string[python]
Attachment Author Key               string[python]
dtype: object

,Id,Key,Attachment Id,Attachment Filename,Attachment Created,Attachment Mime Type,Attachment Size,Attachment Thumbnail,Attachment Content,Attachment Author Key
0,634414,FLOW-230,555153,image-2023-02-13-17-52-39-060.png,2023-02-13 17:52:39.265000+10:00,image/png,252529.0,https://jira.budgetdirect.com.au/secure/thumbn...,https://jira.budgetdirect.com.au/secure/attach...,JIRAUSER32832
1,634414,FLOW-230,555759,image-2023-02-15-11-21-27-984.png,2023-02-15 11:21:28.031000+10:00,image/png,27495.0,https://jira.budgetdirect.com.au/secure/thumbn...,https://jira.budgetdirect.com.au/secure/attach...,JIRAUSER32832
2,634414,FLOW-230,555773,image-2023-02-15-11-28-24-889.png,2023-02-15 11:28:24.949000+10:00,image/png,115231.0,https://jira.budgetdirect.com.au/secure/thumbn...,https://jira.budgetdirect.com.au/secure/attach...,JIRAUSER32832
3,634414,FLOW-230,555849,image-2023-02-15-13-39-18-467.png,2023-02-15 13:39:18.530000+10:00,image/png,90256.0,https://jira.budgetdirect.com.au/secure/thumbn...,https://jira.budgetdirect.com.au/secure/attach...,JIRAUSER32832
4,634411,FLOW-228,555154,image-2023-02-13-17-53-09-595.png,2023-02-13 17:53:09.804000+10:00,image/png,252529.0,https://jira.budgetdirect.com.au/secure/thumbn...,https://jira.budgetdirect.com.au/secure/attach...,JIRAUSER32832


In [12]:
#! Versions

from IPython.display import display

if 'goldenDF' not in globals() or goldenDF is None: 
    display("Base data frame not loaded") 
else:
    df:pd.DataFrame = goldenDF.copy(deep = True)
    df = df[["Id", "Key", "Fix Version/s"]]
    df = df[df["Fix Version/s"].map(lambda d: len(d)) > 0]

    if df is None or len(df.index) == 0:	
        display("No results")
    else:
        df = df.explode("Fix Version/s")
        df = df.expand("Fix Version/s", ["id", "name"])

        df = df.sentence_case_columns().convert_dtypes().infer_objects().reset_index(drop=True)
        df["Fix Version/s Id"] = df["Fix Version/s Id"].astype('Int64')

        display(df.dtypes)
        display(df.head())

Id                             Int64
Key                   string[python]
Fix Version/s Id               Int64
Fix Version/s Name    string[python]
dtype: object

,Id,Key,Fix Version/s Id,Fix Version/s Name
0,543534,PPI-187,22509,Testing
1,569366,PPI-605,22620,FY22Q3
2,652670,PPI-1711,22511,E2E Capability Uplift
3,543568,PPI-201,22512,Portfolio
4,543566,PPI-200,22512,Portfolio
